# 4sU-seq data analysis using DESeq2 rlog transformed counts
This Jupyter notebook contains scripts used to plot 4sU-seq rlog transformed count data from DESeq2 for given lists of genes.

# Table of contents
1. [Load packages and files](#load-packages-and-files)
2. [Get counts for genes in lists and calculate z-scores of gene abundance](#get-counts-for-genes-in-lists-and-calculate-z-scores-of-gene-abundance)
3. [Plot box plots of gene abundance with p-value annotations](#plot-box-plots-of-gene-abundance-with-p-value-annotations)
4. [Plot heatmaps of gene abundance](#plot-heatmaps-of-gene-abundance)
5. [Plot line plots of gene abundance](#plot-line-plots-of-gene-abundance)

## Load packages and files
Load required packages and and rlog transformed counts file. Additionally, take the average between replicates and format into a dataframe labeled with sample names and times.

In [ ]:
#plot rlog transformed counts from DESeq2
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#enter p-value cutoff and file information here
pCutoff=0.05
resDir="..\\data"
outDir="..\\analyses"
countsFile=resDir+"\\counts_rlog.csv"
#column names in the counts file
sampleCols=["W_01","W_02","W_11","W_12",
	"W_21","W_22","W_31","W_32",
	"X92_01","X92_02","X92_11","X92_12",
	"X92_21","X92_22","X92_31","X92_32",
	"X70_01","X70_02","X70_11","X70_12",
	"X70_21","X70_22","X70_31","X70_32"
	]
samples=["wt","SoF","Ctrl"]
timesInt=[0,1,2,3]
times=[str(time) for time in timesInt]
sampleKeys=[]
for sample in samples:
	for time in times:
		sampleKeys.append(sample+time+"h")
sampleValues=[]
for i,sample in enumerate(sampleKeys):
	x,y=i*2,(i*2)+1#get pairs of columns based on key index (replicates)
	sampleValues.append([sampleCols[x],sampleCols[y]])
sampleDict=dict(zip(sampleKeys,sampleValues))

In [ ]:
#average counts between replicates
def averageCounts(file):
	df=pd.read_csv(file)
	df.columns.values[0]="ens"
	dfCounts=pd.DataFrame(data={"ens":df["ens"]})
	for key in sampleKeys:
		dfCounts[key]=df[sampleDict[key]].mean(axis=1)
	return(dfCounts)
dfCounts=averageCounts(countsFile)

## Get counts for genes in lists and calculate z-scores of gene abundance
Subset the counts dataframe using a given list of genes and calculate the z-score relative to the other samples and time points.

In [ ]:
#subset counts based on external gene list and calculate z-scores
def subset(df,listFile,nameFile):
	dfList=pd.read_csv(outDir+"\\"+listFile, header=None)
	dfNames=pd.read_csv(outDir+"\\"+nameFile, header=None)
	dfSubset=df[df["ens"].isin(dfList[0])]
	#reorder in case input lists are not in order of the count dataframe
	dfSubset.ens=dfSubset.ens.astype("category")
	dfSubset.ens=dfSubset.ens.cat.set_categories(list(dfList[0]))
	dfSubset=dfSubset.sort_values(["ens"])
	dfSubset["name"]=list(dfNames[0])
	return(dfSubset)
def countZscores(df):
	#calculate z-scores relative to all samples for a given gene
	dfZ=stats.zscore(df[sampleKeys],axis=1)
	dfZ["ens"]=df["ens"]
	dfZ["name"]=df["name"]
	return(dfZ)
def formatDataZ(df,cell,time):#used later for formatting dataframes for plotting
	dict={"z-score":df[cell+time+"h"],
		"Cell line":cell,
		"Time":time
		}
	dfOut=pd.DataFrame(dict)
	return dfOut
#SoF Dex-dep.
SoF3hFile="SoF3h_ens.txt"
SoF3hNameFile="SoF3h_sym.txt"
dfSoF3h=subset(dfCounts,SoF3hFile,SoF3hNameFile)
dfSoF3hZ=countZscores(dfSoF3h)
#SoF Dex-ind.
SoFUpFile="SoFPos_ens.txt"
SoFUpNameFile="SoFPos_sym.txt"
dfSoFUp=subset(dfCounts,SoFUpFile,SoFUpNameFile)
dfSoFUpZ=countZscores(dfSoFUp)
#SoF 3h Rep.
SoF3hDownFile="SoF3hDown_ens.txt"
SoF3hDownNameFile="SoF3hDown_sym.txt"
dfSoF3hDown=subset(dfCounts,SoF3hDownFile,SoF3hDownNameFile)
dfSoF3hDownZ=countZscores(dfSoF3hDown)
#SoF Const. Rep.
SoFDownFile="SoFDown_ens.txt"
SoFDownNameFile="SoFDown_sym.txt"
dfSoFDown=subset(dfCounts,SoFDownFile,SoFDownNameFile)
dfSoFDownZ=countZscores(dfSoFDown)
#100 most differentially expressed genes (wt GR)
top100File="top100_wt3h_ens.txt"
top100NameFile="top100_wt3h_sym.txt"
dfTop100=subset(dfCounts,top100File,top100NameFile)
dfTop100Z=countZscores(dfTop100)

## Plot box plots of gene abundance with p-value annotations
Plot the z-score of gene abundance for each gene in a list at each dexamethasone time point and for each cell line. Additionally, compare samples with Mann-Whitney test and annotate the plots with p-values.

In [ ]:
#box plots with stats
from statannotations.Annotator import Annotator
def boxCountsStats(df, pairs):
	fontsize=7
	mm=0.0393701
	figwidth=60
	figheight=40
	plt.rcParams["font.family"]="Arial"
	plt.rcParams["font.size"]=fontsize
	#format dataframes to include samples and times for plotting
	dfs=[]
	for sample in samples:
		for time in times:
			dfs.append(formatDataZ(df,sample,time))
	#concatenate dataframes
	dfPlot=pd.concat(dfs, ignore_index=True)
	fig,ax=plt.subplots(dpi=300, figsize=(figwidth*mm,figheight*mm),layout="constrained")
	palette=["#6389FF","#FF5C5C","#1AC11B"]
	sns.stripplot(data=dfPlot, ax=ax, x="Time", y="z-score", hue="Cell line", palette=palette,
		dodge=True, alpha=0.63, linewidth=figwidth/100, size=figwidth/30, edgecolor="black", legend=False)
	sns.boxplot(data=dfPlot, ax=ax, x="Time", y="z-score", hue="Cell line", palette=palette,
		linewidth=figwidth/60, fliersize=figwidth/30,
		boxprops={'edgecolor':'black'}, whiskerprops={'color':'black'}, capprops={'color':'black'},
		medianprops={'color':'black'}, flierprops={'markeredgecolor':'black'})
	#annotate with p-values from Mann-Whitney test using the provided sample pairs
	annotator = Annotator(ax, pairs, data=dfPlot, x="Time", y="z-score", hue="Cell line")
	annotator.configure(test='Mann-Whitney',text_format='simple',loc='inside',show_test_name=False)
	annotator.apply_and_annotate()
	ax.set_xlabel("Time with dexamethasone (hours)")
	ax.set_ylabel("z-score of gene abundance")
	#"best" sometimes puts the legend in a bad spot
	#ax.legend(loc="upper left", fontsize=fontsize-1)
	ax.legend(loc='best', fontsize=fontsize-1)
	plt.show()
#define sample pairs for statistical tests
SoF3hpairs=[(("0", "wt"), ("0", "SoF")),(("0", "SoF"), ("0", "Ctrl")),(("3", "wt"), ("3", "SoF")),(("3", "SoF"), ("3", "Ctrl"))]
SoFUppairs=[(("0", "SoF"), ("3", "SoF")),(("3", "SoF"), ("3", "Ctrl"))]
boxCountsStats(dfSoF3hZ,SoF3hpairs)
boxCountsStats(dfSoFUpZ,SoFUppairs)
boxCountsStats(dfSoF3hDownZ,SoF3hpairs)
boxCountsStats(dfSoFDownZ, SoF3hpairs)
boxCountsStats(dfTop100Z,SoF3hpairs)

## Plot heatmaps of gene abundance
Plot heatmaps using the z-score of gene abundance for each gene in a list at each dexamethasone time point and for each cell line.

In [ ]:
#heatmap
def heatmapCounts(df):
	plt.rcParams["font.family"]="Arial"
	xlabels=[]
	for sample in samples:
		for time in times:
			xlabels.append(sample+" "+time+"h")
	dfPlot=df[sampleKeys]
	sns.set(rc={"figure.dpi":300, 'savefig.dpi':300})
	heatmap=sns.clustermap(data=dfPlot,col_cluster=False,xticklabels=xlabels,
		yticklabels=df["name"],row_cluster=True,dendrogram_ratio=0.0000001,figsize=(6,6),
		cbar_kws={'label': 'z-score of gene abundance'})
	#attempt to scale gene name fontsize based on how many genes there are
	if len(df.index) >= 50:
		yfontsize=400/len(df.index)
	else:
		yfontsize=12
	heatmap.ax_heatmap.set_yticklabels(heatmap.ax_heatmap.get_ymajorticklabels(), fontsize = yfontsize)
	heatmap.fig.subplots_adjust(right=0.57)
	#you might have to manually change the x position of this color bar to not overlap with gene names
	heatmap.ax_cbar.set_position((0.7, .2, .03, .4))
	plt.show()
	sns.reset_defaults()
heatmapCounts(dfSoF3hZ)
heatmapCounts(dfSoFUpZ)
heatmapCounts(dfSoF3hDownZ)
heatmapCounts(dfSoFDownZ)
heatmapCounts(dfTop100Z)

## Plot line plots of gene abundance
Plot the z-score of gene abundance over time for SoF Dex-dep. and SoF Dex-ind. gene sets as line plots. Dots indicate medians.

In [ ]:
#plot lines over time for models
def lineCounts(df):
	plt.rcParams["font.family"]="Arial"
	dfs=[]
	for sample in samples:
		for time in times:
			dfs.append(formatDataZ(df,sample,time))
	dfPlot=pd.concat(dfs, ignore_index=True)
	fig,ax=plt.subplots(dpi=400)
	palette=["#6389FF","#FF5C5C","#1AC11B"]
	#plot medians as dots with lines connecting
	sns.pointplot(data=dfPlot, x="Time", y="z-score", hue="Cell line", palette=palette,
		estimator=np.median)
	ax.set_xlabel("Time with dexamethasone (hours)")
	ax.set_ylabel("z-score of gene abundance")
	plt.legend(loc="best")
lineCounts(dfSoF3hZ)
lineCounts(dfSoFUpZ)